# Allstate Purchase Predictions
As a customer shops an insurance policy, heshe will receive a number of quotes with different coverage options before purchasing a plan. This is represented in this challenge as a series of rows that include a customer ID, information about the customer, information about the quoted policy, and the cost. Your task is to predict the purchased coverage options using a limited subset of the total interaction history. If the eventual purchase can be predicted sooner in the shopping window, the quoting process is shortened and the issuer is less likely to lose the customer's business.

### Files
The training and test sets contain transaction history for customers that ended up purchasing a policy. For each customer_ID, you are given their quote history. In the training set you have the entire quote history, the last row of which contains the coverage options they purchased. In the test set, you have only a partial history of the quotes and do not have the purchased coverage options. These are truncated to certain lengths to simulate making predictions with less history (higher uncertainty) or more history (lower uncertainty).

For each customer_ID in the test set, you must predict the seven coverage options they end up purchasing.

### What is a customer?
Each customer has many shopping points, where a shopping point is defined by a customer with certain characteristics viewing a product and its associated cost at a particular time.

Some customer characteristics may change over time (e.g. as the customer changes or provides new information), and the cost depends on both the product and the customer characteristics.
A customer may represent a collection of people, as policies can cover more than one person.
A customer may purchase a product that was not viewed!

### Product Options
Each product has 7 customizable options selected by customers, each with 2, 3, or 4 ordinal values possible:

### Variables
1. customer_ID - A unique identifier for the customer
2. shopping_pt - Unique identifier for the shopping point of a given customer
3. record_type - 0=shopping point, 1=purchase point
4. day - Day of the week (0-6, 0=Monday)
5. time - Time of day (HH:MM)
6. state - State where shopping point occurred
7. location - Location ID where shopping point occurred
8. group_size - How many people will be covered under the policy (1, 2, 3 or 4)
9. homeowner - Whether the customer owns a home or not (0=no, 1=yes)
10. car_age - Age of the customer’s car
11. car_value - How valuable was the customer’s car when new
12. risk_factor - An ordinal assessment of how risky the customer is (1, 2, 3, 4)
13. age_oldest - Age of the oldest person in customer's group
14. age_youngest - Age of the youngest person in customer’s group
15. married_couple - Does the customer group contain a married couple (0=no, 1=yes)
16. C_previous - What the customer formerly had or currently has for product option C (0=nothing, 1, 2, 3,4)
17. duration_previous -  how long (in years) the customer was covered by their previous issuer
18. A,B,C,D,E,F,G - the coverage options
19. cost - cost of the quoted coverage options



In [1]:
# set up environment
import sys
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
# READ files
dir = 'C:\\Users\\Lenovo\\PycharmProjects\\Kaggle\\Project3_AllState\\'
train = pd.read_csv(dir + 'train.csv.zip')
test = pd.read_csv(dir + 'test_v2.csv.zip')

## Intro summaries and data exploration
1. common columns
2. shape of both sets
3. null values
4. unique values
5. Data Types
6. Correlation between columns

In [3]:
train.shape, test.shape

((665249, 25), (198856, 25))

In [4]:
# COMMON COLUMNS
if list(train.columns) == list(test.columns):
    print('Both data set have the same columns')

pd.Series(train.columns)

Both data set have the same columns


0           customer_ID
1           shopping_pt
2           record_type
3                   day
4                  time
5                 state
6              location
7            group_size
8             homeowner
9               car_age
10            car_value
11          risk_factor
12           age_oldest
13         age_youngest
14       married_couple
15           C_previous
16    duration_previous
17                    A
18                    B
19                    C
20                    D
21                    E
22                    F
23                    G
24                 cost
dtype: object

In [5]:
# % OF NULL VALUES
setA = (train.isnull().sum() / train.shape[0]) * 100
setB = (test.isnull().sum() / test.shape[0]) * 100
pd.concat([setA, setB], join='outer', axis=1, keys=('Train', 'Test')).query('Train !=0 | Test !=0')

,Train,Test
location,0.000000,0.340950
car_value,0.230139,0.371626
risk_factor,36.139551,37.960635
C_previous,2.812631,4.912600
duration_previous,2.812631,4.912600


In [6]:
# # OF UNIQUE VALUES IN EACH COLUMN
setA = train.apply(lambda x: x.value_counts().count())
setB = test.apply(lambda x: x.value_counts().count())
pd.concat([setA, setB], axis=1, keys=('Train', 'Test'))

,Train,Test
customer_ID,97009,55716
shopping_pt,13,11
record_type,2,1
day,7,7
time,1204,1045
state,36,36
location,6248,6029
group_size,4,4
homeowner,2,2
car_age,67,57


In [8]:
# UNIQUE VALUES IN EACH OF COLUMN
column_list = []
def col_to_list(x):
    if x.value_counts().count() < 20:
        column_list.append(x.name)
train.apply(col_to_list)

unique_values = pd.DataFrame({'Column': [None], 'Values': [None]}).iloc[:,[1,0]]

for col in column_list:
    temp_df = pd.DataFrame({'Column': col, 'Values': [train.loc[:, col].unique()]})
    unique_values = pd.concat([unique_values, temp_df], axis=0)
unique_values

,Column,Values
0,None,None
0,shopping_pt,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]"
0,record_type,"[0, 1]"
0,day,"[0, 3, 4, 2, 1, 5, 6]"
0,group_size,"[2, 1, 3, 4]"
0,homeowner,"[0, 1]"
0,car_value,"[g, e, c, d, f, nan, h, i, b, a]"
0,risk_factor,"[3.0, 4.0, nan, 2.0, 1.0]"
0,married_couple,"[1, 0]"
0,C_previous,"[1.0, 3.0, 2.0, 4.0, nan]"


In [9]:
# DATA TYPES
pd.concat([train.dtypes, test.dtypes], axis=1, keys=['Train', 'Test'])

,Train,Test
customer_ID,int64,int64
shopping_pt,int64,int64
record_type,int64,int64
day,int64,int64
time,object,object
state,object,object
location,int64,float64
group_size,int64,int64
homeowner,int64,int64
car_age,int64,int64


In [349]:
# CORRELATION BETWEEN COLUMNS
corr = train.corr()
corr.to_csv(dir + 'Correlations.csv', index_label='Columns')
corr.iloc[5:14, 14:21].round(3) * 100

,A,B,C,D,E,F,G
group_size,1.8,2.8,8.8,4.6,2.3,1.0,4.8
homeowner,4.8,3.4,18.4,10.9,8.1,3.1,9.5
car_age,-43.7,-8.9,-16.7,-13.8,-27.8,-24.9,-8.4
risk_factor,-6.0,-1.2,-16.3,-13.5,-10.4,6.5,-19.3
age_oldest,0.4,2.5,15.7,8.4,10.0,-1.2,14.2
age_youngest,0.5,1.8,15.0,8.2,9.9,-0.7,13.4
married_couple,2.3,2.1,9.2,4.9,3.3,-0.1,4.5
C_previous,12.3,3.3,65.4,41.1,11.1,-3.4,9.9
duration_previous,3.0,0.7,15.6,10.3,7.5,-1.6,15.3


## Data Manipulation and Cleaning:
1. Join both sets for easier manipulation
* Car_Value change to int
* Time to datetime.Time object
* Day to datetime.Day object
* Try to save few columns as ordered scale
* 

In [10]:
# JOIN BOTH SETS FOR EASIER MANIPULATION
train['Set_No'] = 0
test['Set_No'] = 1
total = pd.concat([train, test], ignore_index=True)

In [11]:
# CONVERT CAR VALUE TO INTEGER
def car_value_converter(x):
    if x == 'a':
        return 1
    elif x == 'b':
        return 2
    elif x == 'c':
        return 3
    elif x == 'd':
        return 4
    elif x == 'e':
        return 5
    elif x == 'f':
        return 6
    elif x == 'g':
        return 7
    elif x == 'h':
        return 8
    else:
        return np.nan
    

total.car_value = total.car_value.apply(car_value_converter)

In [12]:
# CONVERT TIME INTO DATETIME.TIME OBJECT
total.time = total.time.apply(lambda x: dt.strptime(x, '%H:%M').time())

In [208]:
# CONVERT day into DAY format

In [13]:
# FILL LOCATION WITH THE MOST POPULAR LOCATION
fill_loc = total.location.value_counts().index[0]
total.location.fillna(fill_loc, inplace=True)

In [14]:
# FILL CAR VALUE WITH THE MOST POPULAR CAR VALUE
fill_car = total.car_value.value_counts().index[0]
total.car_value.fillna(fill_car, inplace=True)

C_Previous has total of 28.840 missing values what constitutes 2.8% and 4.9% of rows in training and test datasets. Therefore we need to find out a scientific way to fill the null values as it might be an important feature in predicting C. 
1. First approach would be fill it with existing C. But historically previous_C = final C only in 71.5% of cases


In [44]:
# FILL C_PREVIOUS NULLS BY existing C

# find unique customer ID of those who have null previous C
C_previous_nulls = total[total.C_previous.isnull()]
C_previous_C_ID = C_previous_nulls.customer_ID.unique()
total.query('customer_ID in @C_previous_C_ID').to_csv(dir + 'C_previous_nulls.csv', index=False, header=True)
# fill na
# total.C_previous.fillna(total.C, inplace=True)

## DATA Analysis
1. How often final C is C_previous?


In [20]:
# DIVIDE SETS As previously
train_set = total.query('Set_No == 0')
test_set = total.query('Set_No == 1')

C Analysis
1. 71.5% of time final C is previous C (print it as %) - therefore is understanding the rest 28%.
* Using other approach 60.2% of C have the same C as in the step1.
* Table which shows how C was changing over shopping steps for intuitive thoughts. The last option is bought by customer!


In [350]:
# 1)
(train_set.query('record_type == 1')['C_previous'] == train_set.query('record_type == 1')['C']).sum() \
/ train_set.query('record_type == 1')['C'].count()
# 2)
c_set = train_set
c_start = c_set.query('record_type == 0 & shopping_pt == 1').loc[:, ['customer_ID', 'C']]
c_start.columns = ['customer_ID', 'C_start']
c_end = c_set.query('record_type == 1').loc[:, ['customer_ID', 'C']]
c_end.columns = ['customer_ID', 'C_end']
c_merged = pd.merge(c_start, c_end, how='outer', )
(c_merged.C_start == c_merged.C_end).sum() / c_merged.C_start.count()
# 3)
train_set.pivot_table('C', index=['customer_ID', 'shopping_pt']).unstack()


customer_ID          0.001958
shopping_pt         -0.023635
record_type          0.001428
day                 -0.009100
location             0.030187
group_size           0.047601
homeowner            0.094525
car_age             -0.084395
risk_factor         -0.192723
age_oldest           0.141886
age_youngest         0.133767
married_couple       0.045304
C_previous           0.099414
duration_previous    0.153097
A                    0.045335
B                    0.024826
C                    0.107925
D                    0.103398
E                    0.102273
F                    0.071318
G                    1.000000
cost                -0.011274
Set_No                    NaN
Name: G, dtype: float64

G Analysis
1. Only 56% of final G is the same as 1st step. Therefore there is a room for improvement here.
* Show by Age - The older is the mean age, the more expensive is option
* Show G by State - there are states like FL in which people choose only 3 in big margin. 
* Which coumns correlates with G: risk_factor (-19%), duration_previous (15%), age_youngest, age_oldest, C, D, E


In [335]:
# 1)
setA = train_set.query('shopping_pt == 1')['G'].reset_index(drop=True)
setB = train_set.query('record_type == 1')['G'].reset_index(drop=True)
(setA == setB).sum() /train_set.query('record_type == 1')['G'].count()      #0.5592573884897277
# 2)
setA = train_set.query('record_type == 1')
setA['age_mean'] = (setA.age_oldest + setA.age_youngest) / 2
setA.iloc[:, [23, 26]].groupby('G').describe()
# 3) 
train_set.query('record_type == 1').groupby(['state', 'G'])['customer_ID'].count().unstack()
# 4)
train.corr()['G']


C:\Users\Lenovo\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


G,1,2,3,4
state,,,,
AL,899.0,1941.0,494.0,261.0
AR,723.0,674.0,103.0,69.0
CO,1358.0,1393.0,690.0,99.0
CT,1974.0,679.0,82.0,101.0
DC,139.0,193.0,114.0,94.0
DE,227.0,225.0,88.0,57.0
FL,NaN,NaN,11696.0,2909.0
GA,213.0,652.0,273.0,132.0
IA,94.0,459.0,109.0,27.0


In [263]:
setA.groupby('homeowner')['G'].count()
setA.groupby(['age_oldest', 'G'])['customer_ID'].count().unstack()
#setA.groupby('homeowner')['customer_ID'].count()
print(setA.groupby('G')['customer_ID'].count())
setA.groupby(['married_couple', 'G'])['customer_ID'].count().unstack()
setA.to_csv(dir + 'Train_record_type1.csv', index=False)

G
1    20257
2    37778
3    30697
4     8277
Name: customer_ID, dtype: int64


Additional check. Whether example submission file has the same customer ID as in the test file. 
And it has 100% 

In [351]:
samp_sub = pd.read_csv(dir + 'sampleSubmission.csv')
(pd.Series(test_set.customer_ID.unique()) == pd.Series(samp_sub.customer_ID.unique())).sum() \
/ max(len(test_set.customer_ID.unique()), len(samp_sub.customer_ID.unique()))

1.0

## Predicting approaches
1. Simplest approach: Final option == First option  (Result: min=56%, max=75%, mean=)
* Same as above but when possible use C_previous instead of C from point 1 (Result: min=56, max=72%, mean=)
* Predict G only by the state of customer (result: 34% vs 56% previously)
* Use Final option as dependent variable and customer charcteristics as independent. Predict useing only 1st step. 


In [195]:
# METHOD1: FIRST OPTION AS FINAL OPTION
# Calculate the prediction accuracy
opt_predict = train_set.query('shopping_pt == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
opt_predict.reset_index(drop=True, inplace=True)
opt_final = train_set.query('record_type == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
opt_final.reset_index(drop=True, inplace=True)
predict_stat = pd.DataFrame({'Option': [None], 'Prediction': [None]})
for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
    temp_df = pd.DataFrame({'Option': [col], 'Prediction': [(opt_predict[col] == opt_final[col]).sum() / opt_final[col].count()]})
    predict_stat = pd.concat([predict_stat, temp_df])
predict_stat.dropna(inplace=True)

# Use this method to predict Test set
opt_predict_method1 = test_set.query('shopping_pt == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
plan_ID = opt_predict_method1['A'].astype('str')
for col in ['B', 'C', 'D', 'E', 'F', 'G']:
    plan_ID += opt_predict_method1[col].astype('str')
opt_predict_method1 = pd.concat([opt_predict['customer_ID'], plan_ID], axis=1, ignore_index=True)
opt_predict_method1.columns = ['customer_ID', 'plan']
opt_predict_method1.to_csv(dir + 'opt_predict_method1.csv', index=False)

# print result table
predict_stat

,Option,Prediction
0,A,0.644569
0,B,0.718119
0,C,0.601759
0,D,0.696533
0,E,0.702337
0,F,0.631642
0,G,0.559257


In [313]:
# METHOD2: FIRST OPTION AS FINAL OPTION BUT C_PREVIOUS
# Calculate the prediction accuracy
opt_final = train_set.query('record_type == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
opt_final.reset_index(drop=True, inplace=True)

opt_predict = train_set.query('shopping_pt == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
C_previous_df = pd.DataFrame({'C_previous': train_set[train_set.customer_ID != np.nan].groupby('customer_ID')['C_previous'].min()})
opt_predict = pd.merge(opt_predict, C_previous_df, how='left', left_on='customer_ID', right_index=True)
opt_predict.C_previous.fillna(opt_predict.C, inplace=True)
opt_predict.C = opt_predict.C_previous
opt_predict.drop('C_previous', axis=1, inplace=True)
opt_predict.C = opt_predict.C.astype('int', inplace=True)
opt_predict.reset_index(drop=True, inplace=True)

predict_stat = pd.DataFrame({'Option': [None], 'Prediction': [None]})
for col in ['A', 'B', 'C', 'D', 'E', 'F', 'G']:
    temp_df = pd.DataFrame({'Option': [col], 'Prediction': [(opt_predict[col] == opt_final[col]).sum() / opt_final[col].count()]})
    predict_stat = pd.concat([predict_stat, temp_df])
predict_stat.dropna(inplace=True)

# Use this method on Test Set
opt_predict = test_set.query('shopping_pt == 1')[['customer_ID', 'A', 'B', 'C', 'D', 'E', 'F', 'G']]
C_previous_df = pd.DataFrame({'C_previous': test_set[test_set.customer_ID != np.nan].groupby('customer_ID')['C_previous'].min()})
opt_predict = pd.merge(opt_predict, C_previous_df, how='left', left_on='customer_ID', right_index=True)
opt_predict.C_previous.fillna(opt_predict.C, inplace=True)
opt_predict.C = opt_predict.C_previous
opt_predict.drop('C_previous', axis=1, inplace=True)
opt_predict.C = opt_predict.C.astype('int', inplace=True)

# Copy actual best prediction
actual_best = opt_predict.copy()

plan_ID = opt_predict['A'].astype('str')
for col in ['B', 'C', 'D', 'E', 'F', 'G']:
    plan_ID += opt_predict[col].astype('str')
opt_predict_method2 = pd.concat([opt_predict['customer_ID'], plan_ID], axis=1, ignore_index=True)
opt_predict_method2.columns = ['customer_ID', 'plan']
opt_predict_method2.to_csv(dir + 'opt_predict_method2.csv', index=False)
    
# print result table
predict_stat


,Option,Prediction
0,A,0.644569
0,B,0.718119
0,C,0.718748
0,D,0.696533
0,E,0.702337
0,F,0.631642
0,G,0.559257


In [329]:
# METHOD3: PREDICTING G BASED ON STATE
# state df show the most common final option for G per state
df_final = train_set.query('record_type == 1')
state_df = pd.DataFrame({'state': [None], 'option': [None]})
for state in df_final.state.unique():
    opt = df_final.query('state == @state').groupby('G')['customer_ID'].count().idxmax()
    temp_df = pd.DataFrame({'state': [state], 'option': [opt]})
    state_df = pd.concat([state_df, temp_df], ignore_index=True)
state_df.dropna(inplace=True)
state_df = state_df.iloc[:, [1, 0]]

# calculate prediction for train set
opt_final = train_set.query('record_type == 1')[['customer_ID', 'G']]
opt_final.reset_index(drop=True, inplace=True)
opt_predict = train_set.query('shopping_pt == 1')[['customer_ID', 'state']]
opt_predict = pd.merge(opt_predict, state_df, how='inner')
opt_predict.drop('state', axis=1, inplace=True)

result_G = (opt_predict['option'] == opt_final['G']).sum() / opt_final['G'].count()
result_G

0.34167963797173456

In [ ]:
# USE FINAL OPTION AS DEPENDENT VARIABLE (DO NOT TAKE SHOPPING PT INTO CALC)
set = train.query('record_type == 1')